In this notebook we give a very brief tutorial which focuses on the `mnms` python user interface. We'll use the recently released `act_dr6.01` products which are available on `NERSC` and at Princeton (`della`). We will first cover how we obtain a noise simulation -- either from disk or on-the-fly. Then, we'll cover how we generate a model -- the (square-root) covariance matrix from which simulations are drawn. Because the raw `act_dr6.01` maps are not yet available on `NERSC`, we won't yet be able to make a model there.

#### ACT DR6.01
We have released a set of noise models and simulations for `act_dr6.01`. The metadata for these products is stored in a config file at `mnms/configs/act_dr6.01_cmbmask.yaml`. Let's take a quick look to know what we're dealing with:

In [ ]:
from mnms import noise_models as nm, utils
from sofind import utils as s_utils
from pixell import enmap
import yaml

config_fn = utils.get_mnms_fn('act_dr6.01_cmbmask.yaml', 'configs')
config_dict = s_utils.config_from_yaml_file(config_fn)
print(yaml.safe_dump(config_dict))

We shouldn't try to understand every detail here --- if you'd like, feel free to read the documentation in `mnms/noise_models.py` for what each individual entry means. But we should understand the broad strokes:

`BaseNoiseModel`
* These are kwargs that configure the base classes of each noise model. For instance, which [sofind](https://github.com/simonsobs/sofind) data model will we load raw data (i.e., maps) from? `act_dr6.01`, of course! Are we using point-source subtracted maps in the models (a.k.a. `srcfree` maps)? Yes, we are. Why do we add `cmbmask` to the noise model configuration filename, if the data are just coming from `act_dr6.01`? Because we specify a `mask_obs_name` --- for reasons discussed in [Atkins et. al. 2023](http://arxiv.org), our noise simulations have a slightly restricted mask relative to the full ACT DR6.01 dataset in order to optimize performance for "CMB" analyses. This mask is specified by the `mask_obs_name`; hence, we denote models configured here as being `cmbmask`ed. 

e.g. `FDWNoiseModel`
* These are kwargs specific to the directional wavelet noise model. For instance, the wavelet log spacing in $\ell$ is $\lambda=1.6$ (see the "scale-discrete" wavelets from the [S2LET](https://arxiv.org/abs/1211.1680) folks).

So if we load an `FDWNoiseModel` instance from this config, it will grab the settings from *both* the `BaseNoiseModel` block and the `FDWNoiseModel` block.

The `TiledNoiseModel` is computationally the lightest, so let's work with it here:

In [ ]:
tnm = nm.TiledNoiseModel.from_config('act_dr6.01_cmbmask', 'pa5a', 'pa5b')

We had to specify which `qids`, or detector sets, for which to build a noise model. Granular information on the `qids` can be found in `sofind`, but we release models for the following pairs of `qids`:
* [`pa4a`, `pa4b`] (i.e., PA4 f150 and f220)
* [`pa5a`, `pa5b`] (i.e., PA5 f090 and f150)
* [`pa6a`, `pa6b`] (i.e., PA6 f090 and f150)

By supplying a pair of frequency bands to the noise model instance, our model will account for correlations between their noise.

#### Getting a noise sim

Let's explore how we might load a noise sim from disk, or generate one on-the-fly. These behaviors are controlled by the `generate` and `check_on_disk` kwargs of the `get_sim` method:

In [ ]:
# By default, generate=True and check_on_disk=True. We try loading a sim from disk first,
# or generating it on-the-fly if it does not exist.
my_sim = tnm.get_sim(split_num=2, sim_num=4, lmax=5400)

Great, that was fast! That's because this sim exists on disk (you can find it, if you like). We only release `sim_num`s from 0 to 29, so calling a higher number will generate a sim on-the-fly:

In [ ]:
my_new_sim = tnm.get_sim(split_num=4, sim_num=1234, lmax=5400)

The first time we run this, it will take some time --- most of it is spent loading auxiliary data from disk that we need before we can draw the sim. This includes loading the (square-root) noise covariance, or noise `model`, as well as the `mask_obs` that will define the footprint for the sim. By default, these objects are stored in memory so that subsequent calls to `get_sim` (for the same `split_num` and `lmax`) are much faster:

In [ ]:
my_newer_sim = tnm.get_sim(split_num=4, sim_num=2468, lmax=5400)

Great, that *was* much faster.

We can force generating a sim on-the-fly if it already exists on disk by passing `check_on_disk=False` (but why would you do that?). Also, we can force only loading a sim from disk, and never generating on-the-fly, e.g. if you made a large batch of sims that you've saved and don't want to compute a second time:

In [ ]:
my_sim = tnm.get_sim(split_num=4, sim_num=29, lmax=5400, generate=False)

How about:

In [ ]:
my_newer_sim = tnm.get_sim(split_num=4, sim_num=2468, lmax=5400, generate=False)

Oops! We got `my_sim` because we've released `sim_num=29`, but we got an error because `sim_num=2468` does not exist on disk, and we have prevented ourselves from generating it on-the-fly with `generate=False`.

We should note a few properties of our sim:

In [ ]:
my_sim.geometry, my_sim.dtype

The sim shape begins with `(2, 1, 3)`, corresponding to `(qid, split, pol)` (we only ever have one sim per split, so the shape of the `split` axis is always 1). Thus, the `f150, U` component of the sim is given by:

In [ ]:
f150_u = my_sim[1, 0, 2]

The shape of the sim is "downgraded" by a factor of 4 relative to the raw ACT DR6.01 data (i.e., its pixels are .0333 degrees, or 2 arcminutes). This is because the Nyquist bandlimit of the noise model (and sim) we specified is `lmax=5400`, which is 4 times lower than that of the raw data (`21600`). Note, we have only released products at this `lmax`: supplying a different `lmax` will require generating that different noise model (and sim) first.

Let's take a look at some sims quickly! We can appreciate the difference between the 3 noise models we implement:

In [ ]:
my_nms = [
    nm.TiledNoiseModel.from_config('act_dr6.01_cmbmask', 'pa4a', 'pa4b'),
    nm.WaveletNoiseModel.from_config('act_dr6.01_cmbmask', 'pa4a', 'pa4b'),
    nm.FDWNoiseModel.from_config('act_dr6.01_cmbmask', 'pa4a', 'pa4b')
]

# we'll compare the f220 Q components 
utils.plot(enmap.enmap([my_nm.get_sim(0, 0, 5400)[1, 0, 2] for my_nm in my_nms]), downgrade=8, ticks=15, range=750, colorbar_label='$\mu$K', colorbar_labelpad=-14)

In order, these are `tiled`, `isotropic wavelet`, and `directional wavelet` sims. The PA4 f220 band is the noisiest in the ACT DR6.01 release, so these noise sims are perhaps the most interesting visually.

#### Getting a noise model

So, how about getting a new noise model, e.g. with a different `lmax`? We will show how we can do this, although, unless you have the raw ACT DR6.01 maps on disk (and these are **not yet public**, so this would only be if you are an ACT member), this will likely raise some `FileNotFoundError`s for you. But, you could use these examples to make noise models (and sims) of your own pre-existing data, if you wanted!

The interface is very similar to `get_sim`:

In [ ]:
# likely we want to write this model to-disk, so we will supply write=True.
# there are also kwargs for whether we want to keep the model in memory (e.g.,
# if we are about to draw sims from it in the same script). see the docs!

tnm.get_model(split_num=6, lmax=8100, write=True)

Again, this likely failed for you, because we haven't yet made public all the ACT DR6.01 data (including sky masks and raw maps). If/when that happens, we will populate the relevant directories with products. You shouldn't have to change anything --- this cell will just work!